In [ ]:
import tensorflow as tf
import numpy as np
import threading

r = np.arange(0.0,100003.0)
raw_data = np.dstack((r,r,r,r))[0]
raw_target = np.array([[1,0,0]] * 100003)

In [ ]:
queue_input_data = tf.placeholder(tf.float32, shape=[20, 4])
queue_input_target = tf.placeholder(tf.float32, shape=[20, 3])

queue = tf.FIFOQueue(capacity=50, dtypes=[tf.float32, tf.float32], shapes=[[4], [3]])

enqueue_op = queue.enqueue_many([queue_input_data, queue_input_target])
dequeue_op = queue.dequeue()

In [ ]:
# capacity = min_after_dequeue + (num_threads + a small safety margin) * batch_size
data_batch, target_batch = tf.train.batch(dequeue_op, batch_size=15, capacity=40)

In [ ]:
def enqueue(sess):
    under = 0
    max = len(raw_data)
    while True:
        print("starting to write into queue")
        upper = under + 20
        print("try to enqueue ", under, " to ", upper)
        if upper <= max:
            curr_data = raw_data[under:upper]
            curr_target = raw_target[under:upper]
            under = upper
        else:
            rest = upper - max
            curr_data = np.concatenate((raw_data[under:max], raw_data[0:rest]))
            curr_target = np.concatenate((raw_target[under:max], raw_target[0:rest]))
            under = rest

        sess.run(enqueue_op, feed_dict={queue_input_data: curr_data,
                                    queue_input_target: curr_target})
        print("added to the queue")
    print("finished enqueueing")

sess = tf.Session()
enqueue_thread = threading.Thread(target=enqueue, args=[sess])
enqueue_thread.isDaemon()
enqueue_thread.start()

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)

In [ ]:
run_options = tf.RunOptions(timeout_in_ms=4000)
curr_data_batch, curr_target_batch = sess.run([data_batch, target_batch], options=run_options)
print(curr_data_batch)

In [ ]:
from PIL.Image import Image

filename_queue = tf.train.string_input_producer(['ds/592b21faac86d83e39276068.jpg']) #  list of files to read

reader = tf.WholeFileReader()
key, value = reader.read(filename_queue)

my_img = tf.image.decode_png(value) # use png or jpg decoder based on your files.

init_op = tf.global_variables_initializer()
sess = tf.InteractiveSession()

with sess.as_default() as sess:
    sess.run(init_op)

# Start populating the filename queue.

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)

for i in range(1): #length of your filename list
    image = my_img.eval() #here is your image Tensor :) 

print(image.shape)
Image.show(Image.fromstring(np.asarray(image)))

coord.request_stop()
coord.join(threads)

In [ ]:
%matplotlib inline

import numpy as np
import skimage.io as io

cat_img = io.imread('ds/592b21faac86d83e39276068.jpg')
io.imshow(cat_img)

In [ ]:
cat_img.

In [ ]:
# Let's convert the picture into string representation
# using the ndarray.tostring() function 
cat_string = cat_img.tostring()
cat_string

In [ ]:
# Now let's convert the string back to the image
# Important: the dtype should be specified
# otherwise the reconstruction will be errorness
# Reconstruction is 1d, so we need sizes of image
# to fully reconstruct it.
reconstructed_cat_1d = np.fromstring(cat_string, dtype=np.uint8)

In [ ]:
reconstructed_cat_1d

In [ ]:
# Here we reshape the 1d representation
# This is the why we need to store the sizes of image
# along with its serialized representation.
reconstructed_cat_img = reconstructed_cat_1d.reshape(cat_img.shape)

In [ ]:
reconstructed_cat_img

In [ ]:
type(cat_img)

In [ ]:
# Let's check if we got everything right and compare
# reconstructed array to the original one.
np.allclose(cat_img, reconstructed_cat_img)

In [ ]:
filename_pairs = ['ds/5919b823f540530d0e75a884.jpg', 'ds/59097f6aac86d80ed56b301a.jpg', 'ds/591f0f297d1a9783237e04e3.jpg']

In [ ]:
%matplotlib inline

# Important: We are using PIL to read .png files later.
# This was done on purpose to read indexed png files
# in a special way -- only indexes and not map the indexes
# to actual rgb values. This is specific to PASCAL VOC
# dataset data. If you don't want thit type of behaviour
# consider using skimage.io.imread()
from PIL import Image
import numpy as np
import skimage.io as io
import tensorflow as tf


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

tfrecords_filename = 'pascal_voc_segmentation.tfrecords'

writer = tf.python_io.TFRecordWriter(tfrecords_filename)

# Let's collect the real images to later on compare
# to the reconstructed ones
original_images = []

for img_path in filename_pairs:
    img = np.array(Image.open(img_path))
    # The reason to store image sizes was demonstrated
    # in the previous example -- we have to know sizes
    # of images to later read raw serialized string,
    # convert to 1d array and convert to respective
    # shape that image used to have.
    height = img.shape[0]
    width = img.shape[1]
    # Put in the original images into array
    # Just for future check for correctness
    original_images.append(img) 
    img_raw = img.tostring()
   
    example = tf.train.Example(features=tf.train.Features(feature={
        'height': _int64_feature(height),
        'width': _int64_feature(width),
        'image_raw': _bytes_feature(img_raw)}))
    
    writer.write(example.SerializeToString())

writer.close()

In [ ]:
reconstructed_images = []

record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

for string_record in record_iterator:
    example = tf.train.Example()
    example.ParseFromString(string_record)
    
    height = int(example.features.feature['height']
                                 .int64_list
                                 .value[0])
    
    width = int(example.features.feature['width']
                                .int64_list
                                .value[0])
    
    img_string = (example.features.feature['image_raw']
                                  .bytes_list
                                  .value[0])
    
    img_1d = np.fromstring(img_string, dtype=np.uint8)
    reconstructed_img = img_1d.reshape((height, width, -1))
    reconstructed_images.append(reconstructed_img)

In [ ]:
# Let's check if the reconstructed images match
# the original images
for original_pair, reconstructed_pair in zip(original_images, reconstructed_images):
    print(np.allclose(original_pair, reconstructed_pair))

In [ ]:
%matplotlib inline

import tensorflow as tf
import skimage.io as io

IMAGE_HEIGHT = 512
IMAGE_WIDTH = 512

tfrecords_filename = 'pascal_voc_segmentation.tfrecords'

def read_and_decode(filename_queue):
    
    reader = tf.TFRecordReader()

    _, serialized_example = reader.read(filename_queue)

    features = tf.parse_single_example(
      serialized_example,
      # Defaults are not specified since both keys are required.
      features={
        'height': tf.FixedLenFeature([], tf.int64),
        'width': tf.FixedLenFeature([], tf.int64),
        'image_raw': tf.FixedLenFeature([], tf.string)
        })

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
    image = tf.decode_raw(features['image_raw'], tf.uint8)
    height = tf.cast(features['height'], tf.int32)
    width = tf.cast(features['width'], tf.int32)
    
    image_shape = tf.stack([height, width, 3])
    image = tf.reshape(image, image_shape)
    
    image_size_const = tf.constant((IMAGE_HEIGHT, IMAGE_WIDTH, 3), dtype=tf.int32)
    # Random transformations can be put here: right before you crop images
    # to predefined size. To get more information look at the stackoverflow
    # question linked above.
    
    resized_image = tf.image.resize_image_with_crop_or_pad(image=image,
                                                           target_height=IMAGE_HEIGHT,
                                                           target_width=IMAGE_WIDTH)
    
    
    images = tf.train.shuffle_batch([resized_image],
                                    batch_size=2,
                                    capacity=30,
                                    num_threads=2,
                                    min_after_dequeue=10)
    
    return images

In [ ]:
import tflearn.data_augmentation  
from tflearn.data_preprocessing import DataPreprocessing as prep

In [ ]:
tflearn.data_augmentation.ImageAugmentation

In [ ]:
filename_queue = tf.train.string_input_producer([tfrecords_filename], num_epochs=10)

# Even when reading in multiple threads, share the filename
# queue.
image = read_and_decode(filename_queue)

# The op for initializing the variables.
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

with tf.Session()  as sess:
    
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # Let's read off 3 batches just for example
    for i in xrange(3):
    
        img = sess.run(image)
        print(img[0, :, :].shape)
        
        print('current batch')
        
        # We selected the batch size of two
        # So we should get two image pairs in each batch
        # Let's make sure it is random

        io.imshow(img[0, :, :])
        io.show()
        
        io.imshow(img[1, :, :])
        io.show()
    
    coord.request_stop()
    coord.join(threads)

In [ ]:
import PIL as Image
import numpy as np

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
           rotation_range=40,
           width_shift_range=0.2,
           height_shift_range=0.2,
           shear_range=0.2,
           zoom_range=0.2,
           horizontal_flip=True,
           zca_whitening=True,
           zca_epsilon=1e-6,
           rescale=True,
           vertical_flip=True,
           fill_mode='nearest')

img = load_img('ds/591f0f297d1a9783237e04e3.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
# the .flow() command below generates batches of randomly transformed images # and saves the results to the `preview/` directory
i=0
for batch in datagen.flow(x, batch_size=1, 
                          save_to_dir='preview', 
                          save_prefix='test', 
                          save_format='jpeg'):
    check = batch
    i += 1
    if i > 2:
        break  # otherwise the generator would loop indefinitely

In [ ]:
check.shape

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'ds/train'
validation_data_dir = 'ds/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

In [ ]:
epsilon = 1.0e-9
def custom_objective(y_true, y_pred):
    '''Crossentropy with prior agency binary vector
    (1)+-----0.8-----+(1)
        \            /
         \          /
          \----0.2-- 
         /  ---0.2---\ 
        /             \
    (0)+-----0.8-----+(1)
    
    '''
    y_pred = T.clip(y_pred, epsilon, 1.0 - epsilon)
    y_pred /= y_pred.sum(axis=-1, keepdims=True)
    cce = T.nnet.categorical_crossentropy(y_pred, y_true)
    return cce